In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def SAIS(A):
  n = len(A)
  buckets = [0] * (max(A) + 2)
  for a in A:
    buckets[a + 1] += 1
  for b in range(1, len(buckets)):
    buckets[b] += buckets[b - 1]
  isL = [1] * n
  for i in reversed(range(n - 1)):
    isL[i] = +(A[i] > A[i + 1]) if A[i] != A[i + 1] else isL[i + 1]

  def induced_sort(LMS):
    SA = [-1] * (n)
    SA.append(n)
    endpoint = buckets[1:]
    for j in reversed(LMS):
      endpoint[A[j]] -= 1
      SA[endpoint[A[j]]] = j
    startpoint = buckets[:-1]
    for i in range(-1, n):
      j = SA[i] - 1
      if j >= 0 and isL[j]:
        SA[startpoint[A[j]]] = j
        startpoint[A[j]] += 1
    SA.pop()
    endpoint = buckets[1:]
    for i in reversed(range(n)):
      j = SA[i] - 1
      if j >= 0 and not isL[j]:
        endpoint[A[j]] -= 1
        SA[endpoint[A[j]]] = j
    return SA

  isLMS = [+(i and isL[i - 1] and not isL[i]) for i in range(n)]
  isLMS.append(1)
  LMS = [i for i in range(n) if isLMS[i]]
  if len(LMS) > 1:
    SA = induced_sort(LMS)
    LMS2 = [i for i in SA if isLMS[i]]
    prev = -1
    j = 0
    for i in LMS2:
      i1 = prev
      i2 = i
      while prev >= 0 and A[i1] == A[i2]:
        i1 += 1
        i2 += 1
        if isLMS[i1] or isLMS[i2]:
          j -= isLMS[i1] and isLMS[i2]
          break
      j += 1
      prev = i
      SA[i] = j
    LMS = [LMS[i] for i in SAIS([SA[i] for i in LMS])]
  return induced_sort(LMS)

def KASAI(A, SA):
  n = len(A)
  rank = [0] * n
  for i in range(n):
    rank[SA[i]] = i
  LCP = [0] * (n - 1)
  k = 0
  acc = 0
  for i in range(n):
    SAind = rank[i]
    if SAind == n - 1:
      continue
    j = SA[SAind + 1]
    while i + k < n and A[i + k] == A[j + k]:
      k += 1
    LCP[SAind] = k
    acc += k
    k -= k > 0
  return acc

def sol() :
  S = input().rstrip()
  if len(S) == 1 : return print(1)
  sa = SAIS(S)
  acc = KASAI(S, sa)
  total = len(S) * (len(S) + 1) // 2 #모든 연속하는 부분 문자열의 개수 = n(n+1)/2
  print(total - acc)
  
sol()

### 자력솔
- 1605(반복 부분문자열)의 풀이를 역으로 생각하면, \
인접한 부분 문자열중 공통되는 접미사를 포함하고 있는 더 큰 길이의 문자열의 LCP array에는 다음과 같은 정보가 들어있는 것으로 생각할 수 있다.
  - LCP[i] = `부분 문자열에서 해당 접미사를 접미사로 두는 부분 문자열의 등장 횟수`
- 따라서 문자열에서 만들 수 있는 서로 다른 부분 문자열의 개수는 다음과 같이 구할 수 있다.
  - `(부분 문자열의 총 개수) - sum(LCP)`
  - `부분 문자열의 총 개수`는 N = 5일때 1+2+3+4+5 ... 이런식의 공차가 1인 등차수열이므로, N(N+1)//2로 구할 수 있다.

### 다른 풀이(kks227)
- 모든 부분 문자열은 어떤 접미사의 접두사이다.
- 매 접미사를 볼 때마다 이 접미사 안에서 여태 등장하지 않은 접두사를 몇개 만들 수 있는지 생각해보자.
  - `a`의 경우 1개 만들 수 있다.
  - `ana` 에서는 `a`를 이미 만들었으므로 `an`, `ana`로 2개 더 만들 수 있다.
  - `anana`는 `anan`, `anana`로 2개 더 만들 수 있다.
  - `banana`는 `b`, `ba`, `ban`, `bana`, `banan`, `banana`로 6개 더 만들 수 있다.
  - `na`는 `n`, `na`로 2개 더 만들 수 있다.
  - `nana`는 `nan`, `nana`로 2개 더 만들 수 있다.
  - 이렇게 만들어진 서로다른 접두사의 개수를 모두 더하면 15가 된다. (1+2+2+6+2+2)
  - 이미 만들어진 접두사의 개수를 매번 걸렀기 때문에 새로운 접두사의 개수만 구할 수 있었고, 이를 누적해서 더해준 것이 답이 된다.
- 따라서 $\displaystyle \sum_{i=1}^{N} (N - SA[i] - LCP[i])$ 가 답이 된다.
  - $N - SA[i]$ 는 이번 접미사의 글자 수이고, 여기에 $LCP[i]$ 를 빼서 이미 등장한 접두사의 개수를 빼준다.
  - 그리고 마지막 접미사에 해당하는 글자수는 없으니 글자수만 더한다.